In [1]:
import geemap
import folium
import ee
import os


import numpy as np
import palettable 
import numpy as np
from Satellite import *
from Simulator import *
service_account = "jims-948@jimssatellites.iam.gserviceaccount.com"
key_path = 'jimssatellites-25b0240d5cf2.json'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials=credentials)


count = 0
import csv
sat1 = Satellite("Satellites/sat1.txt", "SAT1")
sat2 = Satellite("Satellites/sat2.txt", "SAT2")
sat3 = Satellite("Satellites/sat3.txt", "SAT3")
sat4 = Satellite("Satellites/sat4.txt", "SAT4")
sat5 = Satellite("Satellites/sat5.txt", "SAT5")
sat6 = Satellite("Satellites/sat6.txt", "SAT6")  
sat7 = Satellite("Satellites/sat7.txt", "SAT7")
sat8 = Satellite("Satellites/sat8.txt", "SAT8")
sat9 = Satellite("Satellites/sat9.txt", "SAT9")  
sat10 = Satellite("Satellites/sat10.txt", "SAT10")
sat11 = Satellite("Satellites/sat11.txt", "SAT11")
sat12 = Satellite("Satellites/sat12.txt", "SAT12")

satellites = [sat1,sat2,sat3,sat4,sat5,sat6,sat7,sat8,sat9,sat10,sat11,sat12]

count = 0
mappingArray = []
for sat in satellites:
    csv_file = "RESULTS_"+sat.name+".csv"
    count 
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.reader(file)  
        i = 0
        for row in reader:
            coord1a, coord1b,  coord4a, coord4b, error = [float(param) for param in row]
            roi = ee.Geometry.Rectangle([[coord1a, coord1b],  [coord4a, coord4b]])
            newFeature = ee.Feature(roi, {'name': str(count)})
            count = count+1
            mappingArray.append(newFeature)
            count+=1




In [2]:

# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2)

#Satellite mapped area
mappedArea = ee.FeatureCollection(mappingArray)


#NSW bounding box
NSW = ee.Geometry.Rectangle([[144.81577841465915, -36.42295520530404], [155.81727106335805, -27.806830550620564]])


# Create an image collection for Sentinel-2 data
start_date = '2019-06-01'
end_date = '2020-05-31'
sentinel2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                        .filterDate(ee.Date(start_date), ee.Date(end_date))
                        .filterBounds(mappedArea)
                        .select('B.*|MSK_CLDPRB|SCL')
)

#Get the historic burned area dataset
burnedArea = ee.Image('projects/jimssatellites/assets/fires')

#Sentinel2 Short wave infrared
swir1 = sentinel2_collection.select('B11')  # Short-Wave Infrared Band 1 (B11)


##CREATE THE BLACK MASK
roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83])
#I have no idea why, but for some reason this just paints everything so I'm gonna cop for now
mask = ee.Image(0).toInt8().paint(roi, 1)
fc = ee.FeatureCollection(mappingArray)
# Fill and outline the polygons in two colors
filled = mask.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))


nir_palette = ['000000', '330000', '660000', '990000', 'CC0000', 'FF0000', 'FF1900', 'FF3200', 'FF4B00', 'FF6400',
              'FF7D00', 'FF9600', 'FFAF00', 'FFC800', 'FFE100', 'FFFF00']

#Combine the sentinel2 data and Jims satellites data
swir1_mosaic = swir1.mosaic()
swir1_mosaic = swir1_mosaic.visualize(**{'bands': 'B11', 'min': 0,'max': 10000, 'palette': nir_palette})
mask = result.visualize(**{'palette': '000000,FF0000','max': 10000,'opacity': 1.0})
mosaic = ee.ImageCollection([swir1_mosaic, mask]).mosaic()


#Add the historic data to its own mosaic
burnedVisParams = {
    'min': 0,
    'max': 255,
    'palette': [  'fcffa4', 'f2f482', 'f3e35a', 'f8cf3a', 'fbbc21', 'fca80d', 'fb9606',
  'f8850f', 'f2741c', 'eb6429', 'e15635', 'd54a41', 'c83f4b', 'bc3754',
  'ad305d', '9d2964', '8d2369', '7d1e6d', '6d186e', '5d126e', '4f0d6c',
  '3e0966', '2d0b59', '1c0c43', '0e092b', '040314', '000004']
}
roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83], None, False)
burnedAreaViz = burnedArea.visualize(**burnedVisParams)
historic_mosaic = ee.ImageCollection([swir1_mosaic, burnedAreaViz, mask]).mosaic()



In [5]:
from PIL import Image as im 

current_directory = os.getcwd()
out_dir = os.path.join(current_directory)




comleroy = [  150.78448429,-33.38,  151.2087900435, -33.0931176]
yaouk = [148.7428497953762, -35.96021682698345, 149.0150704273652, -35.71831894036284]
piliga =[148.99253089357404,-31.069826237629815, 149.57756512477508, -30.659188247907124]
filename = os.path.join(out_dir, 'piliga_historic.tif')
roi = ee.Geometry.Rectangle(piliga, None, False)
image = historic_mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)


#Visualise the mapped result
Map.addLayer(mosaic, {}, 'Mosaic')
Map.addLayer(historic_mosaic, {}, "historic_mosaic")
Map


Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\piliga_historic.tif


Map(bottom=39018.0, center=[-31.043521630684204, 870.5073341869054], controls=(WidgetControl(options=['positio…